## Higgs dataset preprocessing

From the paper:

[M. De Domenico, A. Lima, P. Mougel and M. Musolesi. The Anatomy of a Scientific Rumor. (Nature Open Access) Scientific Reports 3, 2980 (2013).](http://www.nature.com/srep/2013/131018/srep02980/full/srep02980.html)



In [33]:
import os
import time
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
import itertools


# Customize plot colors for dark backgrounds
%matplotlib inline
mpl.rcParams['axes.edgecolor'] = 'grey'
mpl.rcParams['grid.color'] = '#66CCCC'
mpl.rcParams['text.color'] = '#0EBFE9'
mpl.rcParams['xtick.color'] = '#66CCCC'
mpl.rcParams['ytick.color'] = '#66CCCC'
mpl.rcParams['axes.labelcolor'] = '#0EBFE9'

import IPython.utils.path
DATA_DIR = os.path.join(IPython.utils.path.get_home_dir(), 'data/higgs/')
print 'Data directory:', DATA_DIR
dataset_name = 'higgs'

%load_ext autoreload
%autoreload 2

Data directory: /home/ubuntu/data/higgs/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Parse data

### Parse activity

In [2]:
def create_graph_from_activity(activity_df, action='RT'):
    g = nx.DiGraph()
    df = activity_df[activity_df['action'] == action]
    g.name = 'Higgs ' + action
    for idx, d in df.iterrows():
        src = d['src_id']
        tgt = d['tgt_id']
        if not g.has_edge(src, tgt):
            g.add_edge(src, tgt, weight=1)
        else:
            g[src][tgt]['weight'] += 1

    return g

ACTIVITY = pd.read_csv(os.path.join(DATA_DIR, 'HiggsDiscovery_multiplex_time.txt'), 
                       sep=' ', header=None, names=['src_id', 'tgt_id', 'timestamp', 'action'],
                       dtype={'src_id': np.int64, 'tgt_id': np.int64, 'timestamp': np.int64, 'action': str},
                       index_col=2)

ACTIVITY['action'] = ACTIVITY['action'].astype(str)
ACTIVITY.index = pd.to_datetime(ACTIVITY.index.values * 1e9)
ACTIVITY.index.name = 'timestamp'
# G = create_graph_from_activity(ACTIVITY, 'RT')
# print nx.info(G)

In [29]:
G = create_graph_from_activity(ACTIVITY, 'RE')
print nx.info(G)

Name: Higgs RE
Type: DiGraph
Number of nodes: 39940
Number of edges: 33728
Average in degree:   0.8445
Average out degree:   0.8445


### Parse retweet graph

In [3]:
start = time.time()
# path = os.path.join(DATA_DIR, 'HiggsDiscovery_RT.edges.gz')
# RETWEET = nx.read_edgelist(path, create_using=nx.DiGraph(),
#                            nodetype=int, data=(('weight', int),))
# RETWEET.name = 'Higgs RT'
# nx.write_gpickle(RETWEET, os.path.join(DATA_DIR, 'retweet.gpickle'))
RETWEET = nx.read_gpickle(os.path.join(DATA_DIR, 'retweet.gpickle'))
print 'Retweet graph loaded in:', time.time() - start
print nx.info(RETWEET)

Retweet graph loaded in: 9.10212087631
Name: Higgs RT
Type: DiGraph
Number of nodes: 257827
Number of edges: 334208
Average in degree:   1.2962
Average out degree:   1.2962


### Parse mention graph    

In [25]:
start = time.time()
# MENTION = nx.read_edgelist(os.path.join(DATA_DIR, 'HiggsDiscovery_MT.edges.gz'), 
#                            create_using=nx.DiGraph(), nodetype=int, data=(('weight', int),))
# MENTION.name = 'Higgs MT'
# nx.write_gpickle(MENTION, os.path.join(DATA_DIR, 'mention.gpickle'))

MENTION = nx.read_gpickle(os.path.join(DATA_DIR, 'mention.gpickle'))
print 'Mention graph loaded in:', time.time() - start
print nx.info(MENTION)

Mention graph loaded in: 7.37287902832
Name: Higgs MT
Type: DiGraph
Number of nodes: 118659
Number of edges: 156371
Average in degree:   1.3178
Average out degree:   1.3178


### Parse reply graph

In [28]:
start = time.time()
# REPLY = nx.read_edgelist(os.path.join(DATA_DIR, 'HiggsDiscovery_RE.edges.gz'),
#                          create_using=nx.DiGraph(), nodetype=int, data=(('weight', int),))
# REPLY.name = 'Higgs RE'
# nx.write_gpickle(REPLY, os.path.join(DATA_DIR, 'reply.gpickle'))

REPLY = nx.read_gpickle(os.path.join(DATA_DIR, 'reply.gpickle'))
print 'Reply graph loaded in:', time.time() - start
print nx.info(REPLY)

Reply graph loaded in: 1.68447113037
Name: Higgs RE
Type: DiGraph
Number of nodes: 39940
Number of edges: 33728
Average in degree:   0.8445
Average out degree:   0.8445


### Parse social network (follower network)

In [91]:
start = time.time()
# SOCIAL = nx.read_edgelist(os.path.join(DATA_DIR, 'HiggsDiscovery_social.edges.gz'),
#                           create_using=nx.DiGraph(), nodetype=int, )
# SOCIAL.name = 'Higgs SOCIAL'
# nx.write_gpickle(SOCIAL, os.path.join(DATA_DIR, 'social.gpickle'))

SOCIAL = nx.read_gpickle(os.path.join(DATA_DIR, 'social.gpickle'))
print 'Social graph loaded in:', time.time() - start
print nx.info(SOCIAL)

Social graph loaded in: 56.2548730373
Name: Higgs SOCIAL
Type: DiGraph
Number of nodes: 456626
Number of edges: 14855842
Average in degree:  32.5339
Average out degree:  32.5339


### Extract causal multilayer graph activated components

In [152]:
def create_layered_graph(input_df, freq=1, unit='min', action='RT', social_graph=None):
    
    def round_df(df, freq, unit):
        mul = 1
        if unit == 'min':
            mul = 60
            unit = 'Min'
            
        elif unit == 'hour':
            mul = 60 * 60
            unit = 'H'
            
        elif unit == 'day':
            mul = 60 * 60 * 24
            unit = 'D'
        else:
            mul = 1
            unit = 'S'
        
        ns_min = freq * mul * 1000000000
        idx = pd.DatetimeIndex(((df.index.astype(np.int64) // ns_min) * ns_min))
        df.index = idx
        return df, unit


    def create_layers(df, freq, unit):
        # Trick: use pandas resample to generate continuous indexes
        dfR = df.resample(str(freq) + unit) 
        layer_map = dict(itertools.izip(dfR.index.astype(np.int64), itertools.count()))
        df['layer'] = np.vectorize(lambda x: layer_map[x])(df.index.astype(np.int64))
        # delta between layers (timedelta)
        delta = dfR.index[1] - dfR.index[0]
        return df, delta

    
    h = nx.DiGraph()
    h.name = 'Higgs layered ' + action + ' - ' + str(freq) + ' ' + unit
    
    df = input_df.copy()
    if action != '' and action is not None:
        df = df[df['action'] == action]
    
    df, unit = round_df(df, freq, unit)
    df, delta_ts = create_layers(df, freq, unit)
    # Maximum id in whole dataframe, min should not be 0
    max_id = np.max(np.max(df[['src_id', 'tgt_id']]).values)
    
    for idx, row in df.iterrows():
        base_src = row['src_id']
        base_tgt = row['tgt_id']
        layer = row['layer']
        act = row['action']
        
        if act == 'RT':  # invert edges
            base_src, base_tgt = base_tgt, base_src
        
        src = base_src + (layer * max_id)
        tgt = base_tgt + ((layer + 1) * max_id)
        
        # Add nodes
        h.add_node(src, {'base_id': base_src, 'layer': layer, 'timestamp': idx})
        h.add_node(tgt, {'base_id': base_tgt, 'layer': layer + 1, 'timestamp': idx + delta_ts})
        
        # Add edge
        e_d = dict()
        if social_graph is not None:
            if social_graph.has_edge(base_src, base_tgt):
                e_d['is_social'] = True
            else:
                e_d['is_social'] = False
            
        h.add_edge(src, tgt, e_d)
        
    return h, df


def extract_components(h):
    components = filter(lambda x: x.number_of_edges() >= 1 and x.number_of_nodes() >= 2, 
                    nx.weakly_connected_component_subgraphs(h))
    res = []
    for i, comp in enumerate(components):
        comp.name = i
        res.append({'component': comp})
        
    df = pd.DataFrame(res)
    df.index.name = 'component_id'
    return df


def enrich_components(df, with_social=False):
    
    def get_period_span(c):
        ts = sorted(nx.get_node_attributes(c, 'timestamp').values())
        return (ts[0], ts[-1])
    
    def get_social_edge_ratio(c):
        social_edges = sum(nx.get_edge_attributes(c, 'is_social').values())  # True == 1, False 0
        return social_edges / float(c.number_of_edges())
        
    df['node_count'] = df['component'].apply(lambda x: x.number_of_nodes())
    df['edge_count'] = df['component'].apply(lambda x: x.number_of_edges())
    df['height'] = df['component'].apply(lambda x: len(np.unique(nx.get_node_attributes(x, 'base_id').values())))
    df['width'] = df['component'].apply(lambda x: len(np.unique(nx.get_node_attributes(x, 'layer').values())))
    period_series = df['component'].apply(get_period_span)
    df['start'] = period_series.apply(lambda x: x[0])
    df['end'] = period_series.apply(lambda x: x[1])
    df['social_ratio'] = df['component'].apply(get_social_edge_ratio)
    
    return df.sort('node_count', ascending=False)


def create_activated_components(input_df, freq=1, unit='min', action='RT', social_graph=None):
    h, _ = create_layered_graph(input_df, freq, unit, action, social_graph)
    comp_df = extract_components(h)
    with_social = True if social_graph is not None else False
    return enrich_components(comp_df, with_social)

In [130]:
COMPS = create_activated_components(ACTIVITY, 10, 'min', 'MT', SOCIAL)

In [154]:
COMPS = enrich_components(COMPS, True)
COMPS[COMPS['size'] > 3]

,component,size,height,width,node_count,edge_count,start,end,social_ratio
component_id,,,,,,,,,
17,"(221211307, 220921493, 218857479, 221160318, 2...",18467,12869,31,18467,21620,2012-07-04 03:30:00,2012-07-04 08:30:00,0.514524
62,"(233191424, 233582592, 232071023, 233166856, 2...",891,819,7,891,943,2012-07-04 09:50:00,2012-07-04 10:50:00,0.394486
211,"(240406532, 241124353, 238837769, 241111050, 2...",797,734,8,797,835,2012-07-04 12:10:00,2012-07-04 13:20:00,0.508982
498,"(229695488, 229658625, 229236738, 229779460, 2...",693,636,6,693,747,2012-07-04 08:20:00,2012-07-04 09:10:00,0.488621
1381,"(242849792, 242886657, 242898951, 242464777, 2...",442,418,5,442,465,2012-07-04 13:20:00,2012-07-04 14:00:00,0.619355
374,"(237561856, 238266378, 238243858, 238219285, 2...",421,396,4,421,444,2012-07-04 11:40:00,2012-07-04 12:10:00,0.378378
164,"(235100166, 235558920, 234819601, 235982866, 2...",407,378,4,407,438,2012-07-04 10:40:00,2012-07-04 11:10:00,0.367580
445,"(231505921, 231892995, 231962456, 231090194, 2...",367,338,5,367,379,2012-07-04 09:20:00,2012-07-04 10:00:00,0.490765
1360,"(243378438, 243018758, 243039239, 244140716, 2...",319,307,6,319,342,2012-07-04 13:30:00,2012-07-04 14:20:00,0.567251


## Analysis

### Overlap between retweet and social network

In [4]:
def overlap_graph(g1, g2):
    common_edges = 0
    for u, v in g1.edges_iter():
        if g2.has_edge(u, v):
            common_edges += 1

    res = common_edges * 100 / float(nx.number_of_edges(g1))
    print 'Percentage of overlap (', g1.name, ',', g2.name, '):', res
    print 'Number of common edges:', common_edges
    return res, common_edges

# overlap_graph(REPLY, SOCIAL)
overlap_graph(RETWEET, SOCIAL)
# overlap_graph(MENTION, SOCIAL)

Percentage of overlap ( Higgs RT , Higgs SOCIAL ): 59.1912820758
Number of common edges: 197822


(59.191282075833016, 197822)